In [10]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from sklearn.ensemble import IsolationForest
from sklearn.tree import DecisionTreeClassifier

discharge_path = '/Users/kkkg0829/Desktop/project/data set/B0005_with_lowess_features.csv'
df = pd.read_csv(discharge_path)

# 모델 학습 (cycle_idx 제외)
features = [
    'Voltage_measured', 'Current_measured', 'Temperature_measured',
    'Current_load', 'Voltage_load', 'Voltage_measured_smooth',
    'Voltage_measured_residual', 'Voltage_measured_trend',
    'Current_measured_smooth', 'Current_measured_residual',
    'Current_measured_trend', 'Temperature_measured_smooth',
    'Temperature_measured_residual', 'Temperature_measured_trend',
    'Current_load_smooth', 'Current_load_residual', 'Current_load_trend',
    'Voltage_load_smooth', 'Voltage_load_residual', 'Voltage_load_trend'
]
X = df[features]

iso_model = IsolationForest(contamination=0.01, random_state=42)
df['anomaly'] = iso_model.fit_predict(X)

# 원인 분석 (어떤 피쳐가 이상치에 영향을 줬는가?)
# 이상치(-1)와 정상(1)을 구분하는 Decision Tree를 학습시켜 중요도 추출
explainer = DecisionTreeClassifier(max_depth=4, random_state=42)
explainer.fit(X, df['anomaly'])

# 중요도 DataFrame 생성
importance_df = pd.DataFrame({
    'Feature': features,
    'Importance': explainer.feature_importances_
}).sort_values(by='Importance', ascending=False)

# 시각화
# 정상 데이터와 이상치 데이터 분리
anomalies = df[df['anomaly'] == -1]

fig = go.Figure()

# (1) 전체 흐름 - '전압(Voltage)' 기준
fig.add_trace(go.Scatter(
    x=df.index, 
    y=df['Voltage_measured'],
    mode='lines',
    name='정상 데이터 흐름',
    line=dict(color='lightgrey', width=1.5),
    hoverinfo='skip'
))

# (2) 이상치 (빨간 점)
fig.add_trace(go.Scatter(
    x=anomalies.index, 
    y=anomalies['Voltage_measured'],
    mode='markers',
    name='이상치(Anomaly)',
    marker=dict(color='red', size=7, symbol='x'), 
    # 마우스 올렸을 때 보여줄 정보
    customdata=anomalies[['cycle_idx', 'Temperature_measured', 'Current_measured']],
    hovertemplate=(
        "<b>[이상치 탐지]</b><br>" +
        "시간(Index): %{x}<br>" +
        "전압: %{y:.3f}V<br>" +
        "<b>사이클: %{customdata[0]}</b><br>" +
        "온도: %{customdata[1]:.2f}<br>" +
        "전류: %{customdata[2]:.3f}<extra></extra>"
    )
))

fig.update_layout(
    title='<b>배터리 이상치 탐지 시각화</b> (상위 1% 이상치)',
    xaxis_title='시간 흐름 (Time Steps)',
    yaxis_title='전압 (Voltage)',
    template='plotly_white',
    hovermode='closest',
    height=600
)

fig.show()

In [9]:
# 1. 원인 분석 모델 학습 (Surrogate Model)
explainer = DecisionTreeClassifier(max_depth=4, random_state=42)
explainer.fit(X, df['anomaly'])

# 2. 중요도 데이터프레임 만들기
importance_df = pd.DataFrame({
    'Feature': features,
    'Importance': explainer.feature_importances_
}).sort_values(by='Importance', ascending=True) # 그래프를 위해 오름차순 정렬

# 중요도가 0인(영향 없는) 피쳐는 제거해서 깔끔하게
importance_df = importance_df[importance_df['Importance'] > 0]

# 3. 시각화 (가로 막대 그래프)
fig_imp = px.bar(
    importance_df, 
    x='Importance', 
    y='Feature', 
    orientation='h',
    title='이상치 발생에 영향을 준 주요 원인 (Feature Importance)',
    labels={'Importance': '영향력 (비중)', 'Feature': '피쳐 이름'},
    text_auto='.1%'
)

fig_imp.update_layout(template='plotly_white')
fig_imp.show()